In [9]:
import pandas as pd
import numpy as np
from datetime import datetime

path1 = "../_data/issue_data.parquet"
path2 = "../_data/monthly_ts_data.parquet"

issue_df = pd.read_parquet(path1)
treas_df = pd.read_parquet(path2)

In [16]:
t_mod = treas_df.merge(issue_df, on=['kytreasno', 'kycrspid'], how='left')

In [17]:
t_mod

,kycrspid,kytreasno,mcaldt,tmpubout,tmyld,tmatdt
0,19610622.800000,200001.0,1960-10-31,NaN,0.000076,1961-06-22
1,19610622.800000,200001.0,1960-11-30,NaN,0.000079,1961-06-22
2,19610622.800000,200001.0,1960-12-30,NaN,0.000067,1961-06-22
3,19610622.800000,200001.0,1961-01-31,NaN,0.000068,1961-06-22
4,19610622.800000,200001.0,1961-02-28,NaN,0.000074,1961-06-22
...,...,...,...,...,...,...
192057,20270131.204120,208344.0,2025-01-31,NaN,0.000114,2027-01-31
192058,20280115.204250,208345.0,2025-01-31,NaN,0.000115,2028-01-15
192059,20300131.204250,208346.0,2025-01-31,NaN,0.000117,2030-01-31
192060,20320131.204370,208347.0,2025-01-31,NaN,0.000120,2032-01-31
